In [45]:
import anndata as ad
import pandas as pd
import numpy as np
from numpy.random import default_rng
import scanpy as sc
import scipy
from scipy import sparse

In [46]:
data_path = '../../../../lustre/groups/ml01/datasets/projects/202201212_psngraph_fabiola.curion/data.dir/'
!ls $data_path

all_proteins_V1.txt  combat_rna.h5ad		 cytof_full.h5ad
cite_cells.csv	     cytof_cells.csv		 facs_full.h5ad
combat_adt.h5ad      cytof_cells_harmonised.csv


## Loading adatas

In [47]:
rna = ad.read_h5ad(data_path+"combat_rna.h5ad")
adt = ad.read_h5ad(data_path+"combat_adt.h5ad")
cytof = ad.read_h5ad(data_path+"cytof_full.h5ad")
facs = ad.read_h5ad(data_path+"facs_full.h5ad")

/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/icb/leonardo.golinelli/.conda/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


## Fixing warnings

In [48]:
facs.obs_names = [str(i)+'_facs' for i in range(len(facs))]
adt.var.index = [index[3:] for index in adt.var.index]

In [49]:
rna.shape, adt.shape, cytof.shape, facs.shape

((836148, 20615), (836148, 192), (7118158, 48), (131920, 12))

## Loading label harmonization files

In [50]:
cite_cells = pd.read_csv(data_path+'cite_cells.csv')
cytof_cells = pd.read_csv(data_path+'cytof_cells.csv')
cytof_cells_harmonised = pd.read_csv(data_path+'cytof_cells_harmonised.csv', sep=';')

## Removing original obsm

In [51]:
rna.obsm = None
adt.obsm = None
cytof.obsm = None
facs.obsm = None

## Check variable distributions

In [52]:
#plt.hist(rna.X.data[:100])

In [53]:
#plt.hist(rna.layers['raw'].data[:100])

In [54]:
#plt.hist(adt.X.data[:100])

In [55]:
#plt.hist(adt.layers['raw'].data[:100])

In [56]:
#plt.hist(cytof.X[:,9])

In [57]:
#plt.hist(facs.X[:,1])

In [58]:
#plt.hist(facs.X[:,1])

## Subsampling

In [59]:
#subset size is taken to equal the length of the smallest dataset: i.e. facs
rng = default_rng(seed=100)
rna_subset = rng.choice(len(rna), size=131920, replace=False) 
cytof_subset = rng.choice(len(cytof), size=131920, replace=False)
facs_subset = rng.choice(len(facs), size=131920, replace=False)

In [60]:
#build the query
reference_samples = rna.obs.index[rna_subset] #cells in the training subset to exclude
query_samples = ~rna.obs.index.isin(reference_samples) #samples not in the training subset
rng = default_rng(seed=2)
rna_query_subset = rng.choice(len(query_samples), size=round(131920/4), replace=False)
print("len reference samples: {}".format(len(reference_samples)))
print("len total query samples: {}".format(len(query_samples)))
print("len query subset: {}".format(len(rna_query_subset)))

len reference samples: 131920
len total query samples: 836148
len query subset: 32980


In [61]:
rna_query = rna[rna_query_subset,].copy()
adt_query = adt[rna_query_subset,].copy()
rna = rna[rna_subset,].copy()
adt = adt[rna_subset,].copy() # keep the same observations of the rna assay to provide paired cite seq information
cytof = cytof[cytof_subset,].copy()
facs = facs[facs_subset,].copy()

In [62]:
(rna.obs_names == adt.obs_names).all()

True

In [63]:
rna_query

AnnData object with n_obs × n_vars = 32980 × 20615
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

In [64]:
adt_query

AnnData object with n_obs × n_vars = 32980 × 192
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC',

In [65]:
(rna_query.obs_names == adt_query.obs_names).all()

True

## RNA variable selection and pca

In [66]:
sc.pp.highly_variable_genes(rna, layer = 'raw', n_top_genes=4000, flavor="seurat_v3")
highly_variable_genes = rna.var['highly_variable']
rna = rna[:,highly_variable_genes].copy()
sc.tl.pca(rna, n_comps=50, svd_solver="auto")

## Query RNA variable selection and pca

In [67]:
rna_query

AnnData object with n_obs × n_vars = 32980 × 20615
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

In [68]:
rna_query = rna_query[:,highly_variable_genes].copy()
sc.tl.pca(rna_query, n_comps=50, svd_solver="auto")

In [69]:
rna_query

AnnData object with n_obs × n_vars = 32980 × 4000
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC'

## Use normalized expression as facs.X

In [70]:
facs.layers['original_X'] = facs.X.copy()
facs.X = facs.layers['exprs'].copy()

## Check data matrix sparsification and sparsify

In [71]:
scipy.sparse.issparse(rna.X), scipy.sparse.issparse(adt.X), scipy.sparse.issparse(cytof.X), scipy.sparse.issparse(facs.X)

(True, True, False, False)

In [72]:
cytof.X = sparse.csr_matrix(cytof.X)

In [73]:
facs.X = sparse.csr_matrix(facs.X)

In [74]:
scipy.sparse.issparse(rna.X), scipy.sparse.issparse(adt.X), scipy.sparse.issparse(cytof.X), scipy.sparse.issparse(facs.X)

(True, True, True, True)

## Cell type harmonization

In [75]:
def get_map_raw(l1, l2):
    dic = {}
    for label in l1:
        if label in l2:
            dic.update([(label, label)])
        else:
            dic.update([(label, None)])

    for label in l2:
        if label not in dic:
            dic.update([(label, None)])
    
    return dic

In [76]:
#cite_cells.Annotation_major_subset.unique(), cytof_cells_harmonised.harmonized_major_subset.unique()

In [77]:
#cite_cells.Annotation_cell_type.unique(), cytof_cells_harmonised.harmonized_cell_type.unique()

In [78]:
cell_type_harm_map = get_map_raw(cite_cells.Annotation_cell_type.unique(), cytof_cells_harmonised.harmonized_cell_type.unique())
major_subset_harm_map = get_map_raw(cite_cells.Annotation_major_subset.unique(), cytof_cells_harmonised.harmonized_major_subset.unique())

#### Note: 

From the maps one can see that labels between cytof and citeseq are already harmonized. The only thing to do at this point is to rename the cytof cells as the corresponding harmonized labels present in the cytof_cells_harmonized file.
The only thing to notice is that a subset of cell types identified with one assay are not identified with the other one and vice versa.

In [79]:
#used to map the name present in cytof_cells the same way as harmonized_cytof_cells (and consequently cite_Cells)

dic_major = {}
dic_type = {}
for i in range(len(cytof_cells_harmonised)):
    key = cytof_cells_harmonised.iloc[i]['major_cell_type']
    value_major = cytof_cells_harmonised.iloc[i]['harmonized_major_subset']
    value_type = cytof_cells_harmonised.iloc[i]['harmonized_cell_type']
    if value_major not in dic_major:
        dic_major.update([(key, value_major)])
    if value_type not in dic_type:
        dic_type.update([(key, value_type)])

In [80]:
cytof.obs['harmonized_major_subset'] = cytof.obs['major_cell_type'].map(dic_major)
cytof.obs['harmonized_cell_type'] = cytof.obs['major_cell_type'].map(dic_type)

In [81]:
cytof.obs.rename(columns={'harmonized_major_subset': 'Annotation_major_subset', 'harmonized_cell_type': 'Annotation_cell_type'}, inplace=True)
cytof.obs.columns

Index(['sample_id', 'condition', 'patient_id', 'batch', 'cellID',
       'COMBAT_ID_Time', 'CyTOF_priority', 'major_cell_type',
       'fine_cluster_id', 'Annotation_major_subset', 'Annotation_cell_type'],
      dtype='object')

In [82]:
rna.obs['Annotation_major_subset'].cat.categories

Index(['B', 'CD4', 'CD8', 'DC', 'DN', 'DP', 'GDT', 'HSC', 'MAIT', 'Mast', 'NK',
       'PB', 'PLT', 'RET', 'cMono', 'iNKT', 'nan', 'ncMono'],
      dtype='object')

In [83]:
cytof.obs['Annotation_major_subset'] = cytof.obs['Annotation_major_subset'].astype('category')
cytof.obs['Annotation_cell_type'] = cytof.obs['Annotation_cell_type'].astype('category')

In [84]:
cytof.obs['Annotation_major_subset'] = cytof.obs['Annotation_major_subset'].cat.rename_categories({'UNCLASSIFIED': 'nan'})
cytof.obs['Annotation_cell_type'] = cytof.obs['Annotation_cell_type'].cat.rename_categories({'UNCLASSIFIED': 'nan'})
#cytof.obs['Annotation_major_subset'].unique(), cytof.obs['Annotation_cell_type'].unique()

In [85]:
facs.obs['Annotation_major_subset'] = 'WBCs' #to change to CD4
facs.obs['Annotation_cell_type'] = 'WBCs'

## Assign domain to each adata

In [86]:
rna.obs['Domain'] = 'cite'
adt.obs['Domain'] = 'cite'
rna_query.obs['Domain'] = 'cite'
adt_query.obs['Domain'] = 'cite'
cytof.obs['Domain'] = 'cytof'
facs.obs['Domain'] = 'facs'
rna.obs['Domain_major'] = 'rna'
adt.obs['Domain_major'] = 'adt'
rna_query.obs['Domain_major'] = 'rna'
adt_query.obs['Domain_major'] = 'adt'
cytof.obs['Domain_major'] = 'cytof'
facs.obs['Domain_major'] = 'facs'

## Assign reference or query label to each adata

In [87]:
rna.obs['Framework'] = 'reference'
adt.obs['Framework'] = 'reference'
rna_query.obs['Framework'] = 'query'
adt_query.obs['Framework'] = 'query'
cytof.obs['Framework'] = 'reference'
facs.obs['Framework'] = 'reference'

## Write preprocessed, harmonized files

In [88]:
rna.write("rna-pp-harm-sub.h5ad", compression="gzip")
adt.write("adt-pp-harm-sub.h5ad", compression="gzip")
rna_query.write("rna_query-pp-harm-sub.h5ad", compression="gzip")
adt_query.write("adt_query-pp-harm-sub.h5ad", compression="gzip")
cytof.write("cytof-pp-harm-sub.h5ad", compression="gzip")
facs.write("facs-pp-harm-sub.h5ad", compression="gzip")